In [1]:
import pymc as pm
import numpy as np
import arviz as az
from pymc.math import log, exp

%load_ext lab_black

# Deviance*

Adapted from [Unit 9: deviances.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit9/deviances.odc). The lecture has deviances2.odc, but that file wasn't included in the supporting material. We recreate it below anyways.

Todo:
- add more info about DIC and AIC
- add info about WAIC, link to papers showing advantages
    - https://arxiv.org/abs/1507.04544
    - https://arxiv.org/abs/1004.2316

Associated lecture videos
Unit 9 lessons 1 and 2

In [2]:
t = np.array((1, 1, 2, 2, 3, 4, 4, 5, 5, 8))

## Model 1

In [3]:
with pm.Model() as m:
    gamma = pm.Gamma("gamma", 0.001, 0.001)
    mu = pm.Normal("mu", 0, tau=0.001)
    beta = mu ** (-1 / gamma)  # convert to PyMC parameterization

    f = pm.Deterministic("f", mu * gamma * t ** (gamma - 1) * exp(-mu * t**gamma))
    loglik = pm.Deterministic("loglik", log(f))
    pm.Deterministic("deviance", -2 * pm.math.sum(loglik))

    pm.Weibull("lik", gamma, beta, observed=t)

    trace = pm.sample(
        5000, tune=2000, target_accept=0.99, idata_kwargs=dict(log_likelihood=True)
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [gamma, mu]


Sampling 4 chains for 2_000 tune and 5_000 draw iterations (8_000 + 20_000 draws total) took 11 seconds.
There were 71 divergences after tuning. Increase `target_accept` or reparameterize.


PyMC is not super happy with this model, but the results agree with the professor's.

In [4]:
az.summary(trace, var_names=["mu", "gamma", "deviance"], kind="stats")

,mean,sd,hdi_3%,hdi_97%
mu,0.176,0.114,0.013,0.376
gamma,1.487,0.381,0.796,2.201
deviance,43.388,2.432,40.868,47.806


In [5]:
trace

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats
	> observed_data

In [6]:
az.waic(trace)

/Users/aaron/mambaforge/envs/pymc_env2/lib/python3.11/site-packages/arviz/stats/stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 20000 posterior samples and 10 observations log-likelihood matrix.

          Estimate       SE
elpd_waic   -22.50     2.34
p_waic        1.40        -

There has been a warning during the calculation. Please check the results.

## Model 2

In [7]:
with pm.Model() as m2:
    λ = pm.Gamma("λ", 0.001, 0.001)

    f = pm.Deterministic("f", λ * pm.math.exp(-λ * t))
    loglik = pm.Deterministic("loglik", log(f))
    pm.Deterministic("deviance", -2 * pm.math.sum(loglik))

    pm.Exponential("lik", λ, observed=t)

    trace = pm.sample(2000, idata_kwargs=dict(log_likelihood=True))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [λ]


Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 1 seconds.


In [8]:
az.summary(trace, var_names=["λ", "deviance"], kind="stats")

,mean,sd,hdi_3%,hdi_97%
λ,0.285,0.091,0.132,0.458
deviance,46.077,1.393,45.055,48.648


In [9]:
az.waic(trace, scale="deviance")

Computed from 8000 posterior samples and 10 observations log-likelihood matrix.

              Estimate       SE
deviance_waic    46.52     3.82
p_waic            0.40        -

In [10]:
%load_ext watermark
%watermark -n -u -v -iv -p aesara,aeppl

Last updated: Wed Mar 22 2023

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.9.0

aesara: 2.8.10
aeppl : 0.1.1

pymc : 5.1.2
arviz: 0.14.0
numpy: 1.24.2

